In [1]:
import pandas as pd
import numpy as np

In [2]:
a = pd.read_csv('arrests.csv').drop(columns=['Unnamed: 0'])

* Pre 2013, no predPol

* 2013 predPol was deployed in 3 aREAS:
>North Hollywood, Southwest and Foothill

* 2015 predPol was deployed in 21 Areas:
>77th Street, Central, Devonshire, Foothill, Harbor, Hollenbeck, Hollywood, Mission, Newton, N Hollywood, Northeast, Olympic, Pacific, Rampart, Southeast, Southwest, Topanga, Van Nuys, Wilshire, West LA, and West Valley

In [3]:
# some cleaning

In [4]:
def get_pred(row):
    # check year
    yr = row.loc['Year']
    if yr < 2013:
        return 0
    elif yr >= 2015:
        return 1
    else:
        # check area
        ar = row.loc['Area Name']
        areas = ['North Hollywood', 'Sothwest', 'Foothill']
        if ar in areas:
            return 1
        else:
            return 0       

In [5]:
a['Arrest Date'] = pd.to_datetime(a['Arrest Date'])
a['Year'] = a['Arrest Date'].apply(lambda x: x.year)
a['predPol Deployed'] = a.apply(get_pred, axis=1)

In [60]:
# need for groupby work
a['Total'] = pd.Series([1] * a.shape[0])

In [61]:
a.head()

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location,Year,predPol Deployed,Total
0,191811472,2019-05-03,1700.0,18,Southeast,1802,23,F,B,NaN,NaN,M,653.22 PC,NaN,91ST,FIGUEROA,"(33.9543, -118.2827)",2019,1,1
1,5614161,2019-04-29,1040.0,8,West LA,842,41,M,H,3.0,Robbery,F,211PC,ROBBERY,11600 WILSHIRE BL,NaN,"(34.0508, -118.4592)",2019,1,1
2,5806609,2019-11-23,1830.0,4,Hollenbeck,457,23,F,H,3.0,Robbery,F,211PC,ROBBERY,FICKETT,CINCINNATI,"(34.0477, -118.2047)",2019,1,1
3,5615197,2019-04-30,615.0,6,Hollywood,663,27,M,O,5.0,Burglary,F,459PC,BURGLARY,LA BREA,LEXINGTON,"(34.0907, -118.3384)",2019,1,1
4,5615701,2019-04-30,1100.0,9,Van Nuys,901,2,F,H,NaN,NaN,D,300(B)WIC,NaN,RAYMER,SEPULVEDA BL,"(34.2149, -118.4674)",2019,1,1


In [62]:
# look at columns

In [63]:
a.columns

Index(['Report ID', 'Arrest Date', 'Time', 'Area ID', 'Area Name',
       'Reporting District', 'Age', 'Sex Code', 'Descent Code',
       'Charge Group Code', 'Charge Group Description', 'Arrest Type Code',
       'Charge', 'Charge Description', 'Address', 'Cross Street', 'Location',
       'Year', 'predPol Deployed', 'Total'],
      dtype='object')

In [77]:
a['Year'].value_counts()

2012    163294
2010    162402
2011    157625
2013    152628
2014    139316
2015    126057
2016    117998
2017    107487
2018    104129
2019     93443
2020     19531
Name: Year, dtype: int64

In [16]:
a.isnull().sum()

Report ID                        0
Arrest Date                      0
Time                           197
Area ID                          0
Area Name                        0
Reporting District               0
Age                              0
Sex Code                         0
Descent Code                     0
Charge Group Code            91448
Charge Group Description     91990
Arrest Type Code                 0
Charge                           0
Charge Description           91244
Address                          0
Cross Street                579913
Location                         0
dtype: int64

In [19]:
a.dtypes

Report ID                            int64
Arrest Date                 datetime64[ns]
Time                               float64
Area ID                              int64
Area Name                           object
Reporting District                   int64
Age                                  int64
Sex Code                            object
Descent Code                        object
Charge Group Code                  float64
Charge Group Description            object
Arrest Type Code                    object
Charge                              object
Charge Description                  object
Address                             object
Cross Street                        object
Location                            object
dtype: object

In [ ]:
# arrest rates by race

In [7]:
AR_race = pd.DataFrame(a.groupby('Descent Code').apply(lambda x: len(x) / a.shape[0]), columns=['Arrest Rate'])
AR_race

,Arrest Rate
Descent Code,
A,0.004049
B,0.287360
C,0.000627
D,0.000001
F,0.000701
G,0.000042
H,0.466717
I,0.000211
J,0.000126


In [ ]:
# arrest rates by crime type

In [8]:
type_map = {'D': 'Dependent', 'F': 'Felony', 'I': 'Infraction', 'M': 'Misdemeanor', 'O': 'Other'}

In [9]:
# General Type
a['Arrest Type Code'].value_counts()

M    816003
F    430158
I     57506
O     28931
D     11312
Name: Arrest Type Code, dtype: int64

In [10]:
AR_type = pd.DataFrame(a.groupby('Arrest Type Code').apply(lambda x: len(x)), columns=['Total Arrests by Type'])
AR_type['Arrest Rates by Crime Type'] = AR_type['Total Arrests by Type'].apply(lambda x: x / a.shape[0])
AR_type['Crime Description'] = AR_type.index.map(lambda x: type_map[x])
AR_type.sort_values(by=['Arrest Rates by Crime Type'], ascending=False)

,Total Arrests by Type,Arrest Rates by Crime Type,Crime Description
Arrest Type Code,,,
M,816003,0.607186,Misdemeanor
F,430158,0.320079,Felony
I,57506,0.042790,Infraction
O,28931,0.021527,Other
D,11312,0.008417,Dependent


In [11]:
# More Specific
a['Charge Group Description'].value_counts()

Miscellaneous Other Violations    248213
Narcotic Drug Laws                166308
Driving Under Influence           123262
Drunkeness                        116443
Aggravated Assault                 89111
Larceny                            72118
Other Assaults                     65780
Moving Traffic Violations          54335
Liquor Laws                        52534
Prostitution/Allied                40401
Disorderly Conduct                 37256
Weapon (carry/poss)                32192
Robbery                            27643
Vehicle Theft                      26003
Burglary                           24922
Sex (except rape/prst)             13722
Pre-Delinquency                    11904
Fraud/Embezzlement                  9879
Against Family/Child                7696
Forgery/Counterfeit                 7082
Non-Criminal Detention              6968
Receive Stolen Property             6810
Rape                                3075
Disturbing the Peace                2984
Homicide        

In [12]:
AR_type_2 = pd.DataFrame(a.groupby('Charge Group Description').apply(lambda x: len(x)), columns=['Total Arrests by Charge Group Description'])
AR_type_2['Arrest Rates by Charge Group Description'] = AR_type_2['Total Arrests by Charge Group Description'].apply(lambda x: x / a.shape[0])
AR_type_2.sort_values(by=['Arrest Rates by Charge Group Description'], ascending=False)

,Total Arrests by Charge Group Description,Arrest Rates by Charge Group Description
Charge Group Description,,
Miscellaneous Other Violations,248213,0.184695
Narcotic Drug Laws,166308,0.123749
Driving Under Influence,123262,0.091719
Drunkeness,116443,0.086645
Aggravated Assault,89111,0.066307
Larceny,72118,0.053663
Other Assaults,65780,0.048947
Moving Traffic Violations,54335,0.040431
Liquor Laws,52534,0.039090


In [13]:
# Most Specific
a['Charge Description'].value_counts()

DRUNK DRIVING ALCOHOL/DRUGS                 99867
DRINKING IN PUBLIC                          94120
CORPORAL INJURY ON SPOUSE/COHABITANT/ETC    48285
POSSESSION CONTROLLED SUBSTANCE             46192
FTA AFTER WRITTEN PROMISE                   31735
                                            ...  
WEAR HEADSET/COVERS BOTH EARS                   1
DEALER EMPLOYING UNLICENSED SALESMAN            1
POSS INCOMPL ACCESS CARD W/INT COMPLETE         1
FAIL TO EXHIBIT FISH ON DEMAND OF WARDEN        1
DRAW WATER FROM CLOSED WORKS                    1
Name: Charge Description, Length: 2383, dtype: int64

In [ ]:
# Summary stats on arrests by place and by predPol
# --> does predpol affect arrests

In [14]:
a.columns

Index(['Report ID', 'Arrest Date', 'Time', 'Area ID', 'Area Name',
       'Reporting District', 'Age', 'Sex Code', 'Descent Code',
       'Charge Group Code', 'Charge Group Description', 'Arrest Type Code',
       'Charge', 'Charge Description', 'Address', 'Cross Street', 'Location',
       'Year', 'predPol Deployed'],
      dtype='object')

In [57]:
def get_gb(col):
    return a.groupby(['predPol Deployed', col])

In [64]:
def arrest_rates():
    return lambda x: sum(x) / a.shape[0]

def make_pivot(col):
    return pd.pivot_table(a, values='Total', index=[col],
                    columns=['predPol Deployed'], aggfunc=arrest_rates())

In [59]:
code = get_gb('Descent Code')
arrest_type = get_gb('Arrest Type Code')
group_code = get_gb('Charge Group Description')

In [65]:
make_pivot('Descent Code')

predPol Deployed,0,1
Descent Code,,
A,2.092402e-03,1.956976e-03
B,1.633800e-01,1.239800e-01
C,3.936275e-04,2.336466e-04
D,7.440974e-07,7.440974e-07
F,4.442262e-04,2.567136e-04
G,9.673267e-06,3.199619e-05
H,2.648310e-01,2.018863e-01
I,1.436108e-04,6.771287e-05
J,7.292155e-05,5.283092e-05


In [66]:
make_pivot('Arrest Type Code')

predPol Deployed,0,1
Arrest Type Code,,
D,0.006106,0.002311
F,0.178912,0.141167
I,0.008617,0.034173
M,0.364662,0.242524
O,0.010542,0.010986


In [67]:
make_pivot('Charge Group Description')

predPol Deployed,0,1
Charge Group Description,,
Against Family/Child,0.002167,0.003560
Aggravated Assault,0.030291,0.036016
Burglary,0.010597,0.007947
Disorderly Conduct,0.014256,0.013466
Disturbing the Peace,0.001660,0.000560
Driving Under Influence,0.052847,0.038872
Drunkeness,0.066478,0.020167
Federal Offenses,0.000086,0.000096
Forgery/Counterfeit,0.003297,0.001973


In [ ]:
# Can we combine with crime dataset